Source for this demo notebook: https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb

Some additional comments and prints have been added to this notebook for CS 585

In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

import pandas as pd

# Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
nltk.download('conll2002')

nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/sonjia/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [3]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

print("# train sequences:",len(train_sents))
print("# test  sequences:",len(test_sents))

# train sequences: 8323
# test  sequences: 1517
CPU times: user 1.81 s, sys: 123 ms, total: 1.93 s
Wall time: 2.11 s


Data format:

In [4]:
# show one sequence: each sequence is a list of triples (word, POS, tag):
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [5]:
# showing an additional example sequence:
train_sents[5]

[('Por', 'SP', 'O'),
 ('su', 'DP', 'O'),
 ('parte', 'NC', 'O'),
 (',', 'Fc', 'O'),
 ('el', 'DA', 'O'),
 ('Abogado', 'NC', 'B-PER'),
 ('General', 'AQ', 'I-PER'),
 ('de', 'SP', 'O'),
 ('Victoria', 'NC', 'B-LOC'),
 (',', 'Fc', 'O'),
 ('Rob', 'NC', 'B-PER'),
 ('Hulls', 'AQ', 'I-PER'),
 (',', 'Fc', 'O'),
 ('indicó', 'VMI', 'O'),
 ('que', 'CS', 'O'),
 ('no', 'RN', 'O'),
 ('hay', 'VAI', 'O'),
 ('nadie', 'PI', 'O'),
 ('que', 'PR', 'O'),
 ('controle', 'VMS', 'O'),
 ('que', 'CS', 'O'),
 ('las', 'DA', 'O'),
 ('informaciones', 'NC', 'O'),
 ('contenidas', 'AQ', 'O'),
 ('en', 'SP', 'O'),
 ('CrimeNet', 'NC', 'B-MISC'),
 ('son', 'VSI', 'O'),
 ('veraces', 'AQ', 'O'),
 ('.', 'Fp', 'O')]

In [6]:
# showing words only of previous sequence, in readable format:
" ".join([triple[0] for triple in train_sents[5]])

'Por su parte , el Abogado General de Victoria , Rob Hulls , indicó que no hay nadie que controle que las informaciones contenidas en CrimeNet son veraces .'

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

This is what word2features extracts:

In [8]:
print("Original sequence:")
train_sents[0]

Original sequence:


[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [9]:
print("Features for 1st word in sequence:", train_sents[0][0][0])
sent2features(train_sents[0])[0]

Features for 1st word in sequence: Melbourne


['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 'BOS',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp']

Extract the features from the data:

In [10]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 2.26 s, sys: 221 ms, total: 2.48 s
Wall time: 2.59 s


In [11]:
example_sequence_index = 43  # peek at row 43/sequence in training set 
print(y_train[example_sequence_index])

print("\nFirst 5 features present - each word in example sequence:")
example_feature_sequence = X_train[example_sequence_index]

for idx, feats in enumerate(example_feature_sequence):
    print(idx ,"\t".join(feats[:5])," ...")

['O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O']

First 5 features present - each word in example sequence:
0 bias	word.lower=jaraíz	word[-3:]=aíz	word[-2:]=íz	word.isupper=False  ...
1 bias	word.lower=de	word[-3:]=de	word[-2:]=de	word.isupper=False  ...
2 bias	word.lower=la	word[-3:]=la	word[-2:]=la	word.isupper=False  ...
3 bias	word.lower=vera	word[-3:]=era	word[-2:]=ra	word.isupper=False  ...
4 bias	word.lower=(	word[-3:]=(	word[-2:]=(	word.isupper=False  ...
5 bias	word.lower=cáceres	word[-3:]=res	word[-2:]=es	word.isupper=False  ...
6 bias	word.lower=)	word[-3:]=)	word[-2:]=)	word.isupper=False  ...
7 bias	word.lower=.	word[-3:]=.	word[-2:]=.	word.isupper=False  ...


## Train the model

To train the model, we create pycrfsuite.Trainer, load the training data and call 'train' method. 
First, create pycrfsuite.Trainer and load the training data to CRFsuite:

In [12]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 4.37 s, sys: 102 ms, total: 4.47 s
Wall time: 4.6 s


Set training parameters. We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [13]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

Possible parameters for the default training algorithm:

In [14]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

Train the model:

In [15]:
%%time
trainer.train('conll2002-esp.crfsuite')

CPU times: user 19.8 s, sys: 208 ms, total: 20 s
Wall time: 20.7 s


trainer.train saves model to a file:

In [16]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r--@ 1 sonjia  staff   600K Oct 24 07:43 ./conll2002-esp.crfsuite


We can also get information about the final state of the model by looking at the trainer's logparser. If we had tagged our input data using the optional group argument in add, and had used the optional holdout argument during train, there would be information about the trainer's performance on the holdout set as well. 

In [17]:
trainer.logparser.last_iteration

{'num': 50,
 'scores': {},
 'loss': 14807.577946,
 'feature_norm': 79.110017,
 'error_norm': 1262.912078,
 'active_features': 11346,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.326}

We can also get this information for every step using trainer.logparser.iterations

In [18]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 14807.577946, 'feature_norm': 79.110017, 'error_norm': 1262.912078, 'active_features': 11346, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.326}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [19]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

Let's tag a sentence to see how it works:

In [20]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


## Evaluate the model

In [21]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [22]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 514 ms, sys: 19.8 ms, total: 534 ms
Wall time: 610 ms


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [23]:
print(bio_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-LOC       0.78      0.75      0.76      1084
       I-LOC       0.66      0.60      0.63       325
      B-MISC       0.69      0.47      0.56       339
      I-MISC       0.61      0.49      0.54       557
       B-ORG       0.79      0.81      0.80      1400
       I-ORG       0.80      0.79      0.80      1104
       B-PER       0.82      0.87      0.84       735
       I-PER       0.87      0.93      0.90       634

   micro avg       0.78      0.76      0.77      6178
   macro avg       0.75      0.71      0.73      6178
weighted avg       0.77      0.76      0.76      6178
 samples avg       0.09      0.09      0.09      6178



/Users/sonjia/cs585/session16/.venv_session16/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sonjia/cs585/session16/.venv_session16/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Let's check what classifier learned

In [24]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-ORG  -> I-ORG   8.631963
I-ORG  -> I-ORG   7.833706
B-PER  -> I-PER   6.998706
B-LOC  -> I-LOC   6.913675
I-MISC -> I-MISC  6.129735
B-MISC -> I-MISC  5.538291
I-LOC  -> I-LOC   4.983567
I-PER  -> I-PER   3.748358
B-ORG  -> B-LOC   1.727090
B-PER  -> B-LOC   1.388267
B-LOC  -> B-LOC   1.240278
O      -> O       1.197929
O      -> B-ORG   1.097062
I-PER  -> B-LOC   1.083332
O      -> B-MISC  1.046113

Top unlikely transitions:
I-PER  -> B-ORG   -2.056130
I-LOC  -> I-ORG   -2.143940
B-ORG  -> I-MISC  -2.167501
I-PER  -> I-ORG   -2.369380
B-ORG  -> I-PER   -2.378110
I-MISC -> I-PER   -2.458788
B-LOC  -> I-PER   -2.516414
I-ORG  -> I-MISC  -2.571973
I-LOC  -> B-PER   -2.697791
I-LOC  -> I-PER   -3.065950
I-ORG  -> I-PER   -3.364434
O      -> I-PER   -7.322841
O      -> I-MISC  -7.648246
O      -> I-ORG   -8.024126
O      -> I-LOC   -8.333815


We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [25]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
8.886516 B-ORG  word.lower=efe-cantabria
8.743642 B-ORG  word.lower=psoe-progresistas
5.769032 B-LOC  -1:word.lower=cantabria
5.195429 I-LOC  -1:word.lower=calle
5.116821 O      word.lower=mayo
4.990871 O      -1:word.lower=día
4.910915 I-ORG  -1:word.lower=l
4.721572 B-MISC word.lower=diversia
4.676259 B-ORG  word.lower=telefónica
4.334354 B-ORG  word[-2:]=-e
4.149862 B-ORG  word.lower=amena
4.141370 B-ORG  word.lower=terra
3.942852 O      word.istitle=False
3.926397 B-ORG  word.lower=continente
3.924672 B-ORG  word.lower=acesa
3.888706 O      word.lower=euro
3.856445 B-PER  -1:word.lower=según
3.812373 B-MISC word.lower=exteriores
3.807582 I-MISC -1:word.lower=1.9
3.807098 B-MISC word.lower=sanidad

Top negative:
-1.965379 O      word.lower=fundación
-1.981541 O      -1:word.lower=británica
-2.118347 O      word.lower=061
-2.190653 B-PER  word[-3:]=nes
-2.226373 B-ORG  postag=SP
-2.226373 B-ORG  postag[:2]=SP
-2.260972 O      word[-3:]=uia
-2.384920 O      -1:word.lower

Some observations:

* **8.743642 B-ORG  word.lower=psoe-progresistas** - the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
* **5.195429 I-LOC  -1:word.lower=calle**: "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
* **-3.529449 O      word.isupper=True**, ** -2.913103 O      word.istitle=True **: UPPERCASED or TitleCased words are likely entities of some kind;
* **-2.585756 O      postag=NP** - proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

## What to do next

1. Load 'testa' Spanish data.
2. Use it to develop better features and to find best model parameters.
3. Apply the model to 'testb' data again.

The model in this notebook is just a starting point; you certainly can do better!

#### Added - CS 585 

In [26]:
# why so many? 
print("# state features:", len(tagger.info().state_features))

# state features: 11265


In [27]:
features = list(tagger.info().state_features.keys())
for pair in features[:20]:
    print(pair)

('bias', 'B-LOC')
('bias', 'O')
('bias', 'B-ORG')
('bias', 'B-PER')
('bias', 'I-PER')
('bias', 'B-MISC')
('bias', 'I-ORG')
('bias', 'I-LOC')
('bias', 'I-MISC')
('word.lower=melbourne', 'B-LOC')
('word[-3:]=rne', 'B-LOC')
('word[-2:]=ne', 'O')
('word[-2:]=ne', 'B-ORG')
('word[-2:]=ne', 'B-PER')
('word[-2:]=ne', 'I-ORG')
('word.isupper=False', 'B-LOC')
('word.isupper=False', 'O')
('word.isupper=False', 'B-ORG')
('word.isupper=False', 'B-PER')
('word.isupper=False', 'I-PER')


In [28]:
# show feature categories e.g. "w[-2':]" in 'word[-2:]=ne'
feature_prefixes = set([feat.split("=")[0] for feat, tag in features])
print("# state feature prefixes:", len(feature_prefixes))
feature_prefixes

# state feature prefixes: 21


{'+1:postag',
 '+1:postag[:2]',
 '+1:word.istitle',
 '+1:word.isupper',
 '+1:word.lower',
 '-1:postag',
 '-1:postag[:2]',
 '-1:word.istitle',
 '-1:word.isupper',
 '-1:word.lower',
 'BOS',
 'EOS',
 'bias',
 'postag',
 'postag[:2]',
 'word.isdigit',
 'word.istitle',
 'word.isupper',
 'word.lower',
 'word[-2:]',
 'word[-3:]'}

In [29]:
## What if we INCREASE the L1 penalty? 

trainer_v2 = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer_v2.append(xseq, yseq)
    
trainer_v2.set_params({
    'c1': 10.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})


In [30]:
trainer_v2.train('experiment_2.crfsuite')
tagger_v2 = pycrfsuite.Tagger()
tagger_v2.open('experiment_2.crfsuite')
print("# state features:", len(tagger_v2.info().state_features))


# state features: 1732
